In [12]:
import requests
import json

def getGu():
    url = "https://parking.seoul.go.kr/SearchParking.do"
    try:
        resp = requests.post(url)
        resp.raise_for_status()  # Raise an error for bad status codes

        if 'list' in resp.json():
            gu_list = resp.json()['list']
            gu_dict = {item['gu_code']: item['gu_name'] for item in gu_list}
            return gu_dict
        else:
            return "No data available"

    except requests.RequestException as e:
        return str(e)


def getParking(gu_code=''):
    url = "https://parking.seoul.go.kr/SearchParkingBy.do"
    try:
        resp = requests.post(url, data={'gu_code': gu_code})
        resp.raise_for_status()

        if 'list' in resp.json():
            parking_list = resp.json()['list']

            result_list = [{
                'name': parking['name'],
                'address': parking['address'],
                'longitude': parking['longitude'],
                'latitude': parking['latitude']
            } for parking in parking_list]

            result_dict = {'parking_list': result_list}
            result_json = json.dumps(result_dict, ensure_ascii=False)

            with open('parking_data.json', 'w', encoding='utf-8') as f:
                f.write(result_json)

            return result_json
        else:
            return "No data available"

    except requests.RequestException as e:
        return str(e)


if __name__ == '__main__':
    print(getGu())
    print(getParking('some_gu_code'))  # Replace 'some_gu_code' with an actual code


HTTPSConnectionPool(host='parking.seoul.go.kr', port=443): Max retries exceeded with url: /SearchParking.do (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
HTTPSConnectionPool(host='parking.seoul.go.kr', port=443): Max retries exceeded with url: /SearchParkingBy.do (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))


In [ ]:
import requests
import json
import pandas as pd

def getGu():
    url = "https://parking.seoul.go.kr/SearchParking.do"
    try:
        resp = requests.post(url)
        resp.raise_for_status()

        if 'list' in resp.json():
            gu_list = resp.json()['list']
            return pd.DataFrame(gu_list)
        else:
            return pd.DataFrame()

    except requests.RequestException as e:
        print(e)
        return pd.DataFrame()


def getParking(gu_code=''):
    url = "https://parking.seoul.go.kr/SearchParkingBy.do"
    try:
        resp = requests.post(url, data={'gu_code': gu_code})
        resp.raise_for_status()

        if 'list' in resp.json():
            parking_list = resp.json()['list']
            return pd.DataFrame(parking_list)
        else:
            return pd.DataFrame()

    except requests.RequestException as e:
        print(e)
        return pd.DataFrame()


if __name__ == '__main__':
    print(getGu())
    print(getParking('특정_구_코드'))  # '특정_구_코드'를 실제 구 코드로 대체하세요.


In [9]:
# -*- coding:utf-8 -*-

import requests
import json

url = "http://parking.seoul.go.kr"

gu_list = ["강남구", "강동구", "강북구", "강서구", "관악구", "광진구", "구로구", "금천구", "노원구", "도봉구",
           "동대문구", "동작구", "마포구", "서대문구", "서초구", "성동구", "성북구", "송파구", "양천구",
           "영등포구", "용산구", "은평구", "종로구", "중구", "중랑구"]
def siDo():
    total_list = []
    for i in gu_list:
        p_list = []
        res = requests.post(url, data={"Gu": '전체',
                                    "Dong": "전체",
                                    "Keyword": "장애인"})
        datas = res.json()['res_value']['parking_list']
        for data in datas:
            dic = {}
            dic['name'] = data['parking_name']
            dic['capacity'] = data['capacity']
            dic['addr'] = data['address']
            dic['lng'] = data['position_list'][0]['lng']
            dic['lat'] = data['position_list'][0]['lat']
            dic['add_rates'] = data['add_rates']
            dic['rates'] = data['rates']

            p_list.append(dic)

        total_list.append(p_list)
    total_dic = {"total":total_list}

    result_json = json.dumps(total_dic, ensure_ascii=False)

    with open('seoulparking.json', 'w', encoding='utf-8') as f:
        f.write(result_json)

siDo()

JSONDecodeError: Expecting value: line 6 column 1 (char 10)

In [8]:
# -*- coding:utf-8 -*-

import requests
import pandas as pd

url = "http://parking.seoul.go.kr"

gu_list = ["강남구", "강동구", "강북구", "강서구", "관악구", "광진구", "구로구", "금천구", "노원구", "도봉구",
           "동대문구", "동작구", "마포구", "서대문구", "서초구", "성동구", "성북구", "송파구", "양천구",
           "영등포구", "용산구", "은평구", "종로구", "중구", "중랑구"]

def siDo():
    all_parking_data = []  # 모든 주차장 데이터를 저장할 리스트

    for gu in gu_list:
        try:
            res = requests.post(url, data={"Gu": gu,
                                           "Dong": "전체",
                                           "Keyword": "장애"})
            res.raise_for_status()  # HTTP 에러가 있을 경우 예외를 발생시킵니다.
            datas = res.json()['res_value']['parking_list']
            for data in datas:
                parking_data = {
                    '구명': gu,
                    '주차장명': data['parking_name'],
                    '수용능력': data['capacity'],
                    '주소': data['address'],
                    '경도': data['position_list'][0]['lng'],
                    '위도': data['position_list'][0]['lat'],
                }
                all_parking_data.append(parking_data)
        except requests.RequestException as e:
            print(f"Error fetching data for {gu}: {e}")

    # pandas DataFrame으로 변환
    df = pd.DataFrame(all_parking_data)
    
    # CSV 파일로 저장
    df.to_csv('seoul_disable_parking.csv', index=False, encoding='utf-8-sig')

siDo()


Error fetching data for 강남구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 강동구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 강북구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 강서구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 관악구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 광진구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 구로구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 금천구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 노원구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 도봉구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 동대문구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 동작구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 마포구: Expecting value: line 6 column 1 (char 10)
Error fetching data for 서대문구: Expecting value: line 6 column 1 